In [11]:
# %%
import pandas as pd
import numpy as np

# Load train and test data
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)
df_train.head()


Train shape: (18153, 21)
Test shape: (7780, 20)


,ID,ASI_category,Temperature,Precipitation,Rainfall,Snowfall,Soil_Temperature,Radiation,Wind_Speed,Wind_Gusts,...,Surface_Pressure,Relative_Humidity,Soil_Moisture,Dew_Point,Sunshine_Duration,Cloud_Cover,Precipitation_Hours,Wind_Direction,Weather_Code,Daylight_Duration
0,19554,Moderate,0.931231,0.000912,0.000912,0.0,0.757673,0.879671,0.179293,0.193029,...,0.538056,55,0.546243,17.564597,53252.08,12.136192,1,176.459082,51,58772.52
1,25205,Moderate,0.566323,0.096715,0.096715,0.0,0.291448,0.008913,0.588384,0.532172,...,0.568475,88,0.557803,5.692134,0.00,91.901341,16,232.433005,61,28143.12
2,771,Poor,0.018033,0.000000,0.000000,0.0,0.000000,0.277340,0.247475,0.189008,...,0.706520,78,0.791908,-25.264420,30213.79,18.859670,0,44.688600,3,34621.43
3,1976,Good,0.717541,0.000000,0.000000,0.0,0.635669,0.796709,0.123737,0.134048,...,0.547500,57,0.473988,5.913865,44627.21,38.759757,0,333.640418,3,59192.17
4,14036,Moderate,0.827170,0.001825,0.001825,0.0,0.743855,0.781282,0.343434,0.391421,...,0.546378,50,0.459538,9.661455,45267.17,60.058955,1,86.996954,51,59956.03


In [12]:
# %%
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_train['ASI_category_encoded'] = le.fit_transform(df_train['ASI_category'])


In [13]:
import pandas as pd
import numpy as np

def feature_engineering(df):
    """
    Combined feature engineering function:
    - Includes both base interaction features and advanced meteorological features.
    - Ensures safe numerical operations and consistent encodings.
    """

    df = df.copy()
    df.columns = df.columns.str.replace(' ', '_').str.replace('.', '', regex=False)
    epsilon = 1e-6  # To avoid division by zero

    # --- BASE FEATURES (Your original ones) ---
    df['Temp_Range_Impact'] = df['Temperature'] * df['Soil_Temperature']
    df['Humidity_Moisture'] = df['Relative_Humidity'] * df['Soil_Moisture']
    df['Effective_Radiation'] = df['Radiation'] * (df['Sunshine_Duration'] / (df['Daylight_Duration'] + epsilon))
    df['Total_Precip'] = df['Rainfall'] + df['Snowfall'] + df['Precipitation']
    df['Wind_Intensity'] = df['Wind_Speed'] * df['Wind_Gusts']
    df['Pressure_Humidity_Interaction'] = df['Surface_Pressure'] * df['Relative_Humidity']

    # --- EXTENDED INTERACTIONS ---
    df['Temp_Diff_Air_Soil'] = df['Temperature'] - df['Soil_Temperature']
    df['Temp_Mean'] = (df['Temperature'] + df['Soil_Temperature']) / 2
    df['Temp_Humidity_Index'] = df['Temperature'] * df['Relative_Humidity']
    df['Radiation_Per_Hour'] = df['Radiation'] / (df['Sunshine_Duration'] + epsilon)
    df['Sunshine_Ratio'] = df['Sunshine_Duration'] / (df['Daylight_Duration'] + epsilon)
    df['Wind_Stress'] = df['Wind_Speed'] ** 2
    df['Wind_Ratio'] = df['Wind_Gusts'] / (df['Wind_Speed'] + epsilon)
    df['Humidity_to_Pressure'] = df['Relative_Humidity'] / (df['Surface_Pressure'] + epsilon)
    df['Radiation_to_Temp'] = df['Radiation'] / (df['Temperature'] + epsilon)

    # --- ADVANCED METEOROLOGICAL FEATURES ---
    df['Temperature_Delta'] = df['Temperature'] - df['Dew_Point']
    df['Soil_Temp_Air_Temp_Ratio'] = np.where(
        df['Temperature'].abs() > epsilon,
        df['Soil_Temperature'] / df['Temperature'],
        0
    )
    df['Rainfall_Intensity'] = df['Rainfall'] / (df['Precipitation_Hours'] + epsilon)
    df['Pressure_MSL_Difference'] = df['Pressure_MSL'] - df['Surface_Pressure']
    df['Wind_Gust_Ratio'] = np.where(
        df['Wind_Speed'].abs() > epsilon,
        df['Wind_Gusts'] / df['Wind_Speed'],
        1.0
    )
    df['Radiation_Efficiency'] = np.where(
        df['Radiation'].abs() > epsilon,
        df['Sunshine_Duration'] / df['Radiation'],
        0
    )
    df['Precipitation_Type_Snow'] = np.where(df['Snowfall'] > 0, 1, 0)
    df['Precipitation_Ratio_Rain'] = np.where(
        df['Precipitation'].abs() > epsilon,
        df['Rainfall'] / df['Precipitation'],
        0
    )
    df['Soil_Moisture_RH_Interaction'] = df['Soil_Moisture'] * df['Relative_Humidity']
    df['Cloud_Rain_Interaction'] = df['Cloud_Cover'] * df['Rainfall']

    # --- Stability and Convection ---
    df['Temperature_Inversion_Delta'] = np.abs(df['Soil_Temperature'] - df['Temperature'])
    df['Humidity_Saturated_Index'] = df['Relative_Humidity'] * np.where(
        df['Temperature'].abs() > epsilon,
        df['Dew_Point'] / df['Temperature'],
        0
    )

    # --- Wind Shear / Turbulence ---
    df['Wind_Force_Index'] = df['Wind_Speed'] ** 2
    df['Gust_Delta_Ratio'] = np.where(
        df['Wind_Speed'].abs() > epsilon,
        (df['Wind_Gusts'] - df['Wind_Speed']) / df['Wind_Speed'],
        0
    )

    # --- Daylight & Cloud ---
    df['Daylight_Efficiency'] = np.where(
        df['Daylight_Duration'].abs() > epsilon,
        df['Sunshine_Duration'] / df['Daylight_Duration'],
        0
    )
    df['Cloud_Cover_Inverse'] = 100 - df['Cloud_Cover']

    # --- Cyclic Wind Direction ---
    if 'Wind_Direction' in df.columns:
        wind_rad = np.deg2rad(df['Wind_Direction'])
        df['Wind_Direction_Sin'] = np.sin(wind_rad)
        df['Wind_Direction_Cos'] = np.cos(wind_rad)

    # --- Pressure Trend by ID (conceptual time component) ---
    if 'ID' in df.columns:
        df = df.sort_values(by=['ID'])
        df['Pressure_MSL_Trend'] = df.groupby('ID')['Pressure_MSL'].diff().fillna(0)

    # --- Clean NaNs only for numeric columns ---
    df = df.replace([np.inf, -np.inf], np.nan)
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    # --- Drop duplicate columns ---
    df = df.loc[:, ~df.columns.duplicated()]

    return df

df_train = feature_engineering(df_train)
df_test = feature_engineering(df_test)


In [14]:
# %%
from sklearn.preprocessing import StandardScaler

num_cols = df_train.select_dtypes(include=[np.number]).columns.drop('ASI_category_encoded')

scaler = StandardScaler()
df_train[num_cols] = scaler.fit_transform(df_train[num_cols])
df_test[num_cols] = scaler.transform(df_test[num_cols])


In [15]:
# %%
from xgboost import XGBClassifier
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --- Define features and target ---
X = df_train.drop(['ASI_category', 'ASI_category_encoded'], axis=1)
y = df_train['ASI_category_encoded']

# --- Train XGBoost model for feature importance ---
xgb_model = XGBClassifier(
    random_state=42,
    n_estimators=300,
    learning_rate=0.05,
    max_depth=9,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss',
    n_jobs=-1
)

xgb_model.fit(X, y)

# --- Extract feature importances ---
importances = xgb_model.feature_importances_
feature_importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

# --- Display Top Features ---
print("Most Important Features:")
print(feature_importance_df)

# # --- Plot Feature Importance ---
# plt.figure(figsize=(10, 8))
# plt.barh(feature_importance_df["Feature"][:20][::-1], feature_importance_df["Importance"][:20][::-1])
# plt.xlabel("Importance Score")
# plt.ylabel("Feature")
# plt.title("Top 20 Feature Importances (XGBoost)")
# plt.tight_layout()
# plt.show()


Most Important Features:
                          Feature  Importance
27                      Temp_Mean    0.222537
20              Temp_Range_Impact    0.101560
22            Effective_Radiation    0.093026
1                     Temperature    0.054316
34              Radiation_to_Temp    0.047103
6                       Radiation    0.042572
14              Sunshine_Duration    0.032083
5                Soil_Temperature    0.020690
13                      Dew_Point    0.015868
46       Humidity_Saturated_Index    0.015154
35              Temperature_Delta    0.015104
19              Daylight_Duration    0.014875
26             Temp_Diff_Air_Soil    0.014764
30                 Sunshine_Ratio    0.013029
49            Daylight_Efficiency    0.012452
41        Precipitation_Type_Snow    0.011208
45    Temperature_Inversion_Delta    0.010265
36       Soil_Temp_Air_Temp_Ratio    0.009613
2                   Precipitation    0.009514
43   Soil_Moisture_RH_Interaction    0.009478
12       

In [17]:
# --- Compute cumulative importance ---
feature_importance_df = feature_importance_df.reset_index(drop=True)
feature_importance_df["Cumulative_Importance"] = feature_importance_df["Importance"].cumsum()

# --- Select features covering 97% of importance ---
selected_features = feature_importance_df[feature_importance_df["Cumulative_Importance"] <= 0.97]["Feature"].tolist()

# --- Identify dropped features ---
dropped_features = feature_importance_df[feature_importance_df["Cumulative_Importance"] > 0.97]["Feature"].tolist()

print(f"✅ Selected {len(selected_features)} features (covering 97% importance):")
print(selected_features)

print(f"\n🗑️ Dropped {len(dropped_features)} less important features:")
print(dropped_features)


✅ Selected 47 features (covering 97% importance):
['Temp_Mean', 'Temp_Range_Impact', 'Effective_Radiation', 'Temperature', 'Radiation_to_Temp', 'Radiation', 'Sunshine_Duration', 'Soil_Temperature', 'Dew_Point', 'Humidity_Saturated_Index', 'Temperature_Delta', 'Daylight_Duration', 'Temp_Diff_Air_Soil', 'Sunshine_Ratio', 'Daylight_Efficiency', 'Precipitation_Type_Snow', 'Temperature_Inversion_Delta', 'Soil_Temp_Air_Temp_Ratio', 'Precipitation', 'Soil_Moisture_RH_Interaction', 'Soil_Moisture', 'Radiation_Efficiency', 'Cloud_Rain_Interaction', 'Humidity_Moisture', 'Rainfall', 'Relative_Humidity', 'Radiation_Per_Hour', 'Snowfall', 'Temp_Humidity_Index', 'Rainfall_Intensity', 'Precipitation_Ratio_Rain', 'Precipitation_Hours', 'Cloud_Cover_Inverse', 'Pressure_Humidity_Interaction', 'Pressure_MSL', 'Wind_Speed', 'Humidity_to_Pressure', 'Cloud_Cover', 'Weather_Code', 'ID', 'Wind_Gust_Ratio', 'Surface_Pressure', 'Wind_Gusts', 'Wind_Ratio', 'Wind_Direction_Cos', 'Wind_Direction_Sin', 'Wind_Force_

In [32]:
from sklearn.model_selection import train_test_split

# --- Use selected features from the previous step ---
X = df_train[selected_features]
y = df_train["ASI_category_encoded"]

# --- Split the data ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.1,          # 20% for validation
    stratify=y,             # preserve class balance
    random_state=42
)

print("✅ Data Split Complete:")
print(f"Training Shape   : {X_train.shape}")
print(f"Validation Shape : {X_val.shape}")
print(f"Target Distribution (Train):")
print(y_train.value_counts(normalize=True))


✅ Data Split Complete:
Training Shape   : (16337, 47)
Validation Shape : (1816, 47)
Target Distribution (Train):
ASI_category_encoded
1    0.701108
0    0.172859
2    0.126033
Name: proportion, dtype: float64


In [37]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss, classification_report

# --- Define model ---
xgb_es = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.75,
    colsample_bytree=0.7,
    gamma=1.0,
    min_child_weight=4,
    reg_alpha=1.0,
    reg_lambda=3.0,
    random_state=42,
    n_jobs=-1,
    eval_metric="mlogloss",
    use_label_encoder=False
)

# --- Train with early stopping (new syntax for XGBoost ≥2.0) ---
xgb_es.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=100,
    early_stopping_rounds=50  # ✅ still works for backward compatibility
)

# --- Best iteration ---
best_iter = getattr(xgb_es, "best_iteration", None)
print(f"\n✅ Best iteration (n_estimators used): {best_iter}")

# --- Evaluate ---
train_preds = xgb_es.predict(X_train)
val_preds = xgb_es.predict(X_val)
train_probs = xgb_es.predict_proba(X_train)
val_probs = xgb_es.predict_proba(X_val)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average='macro')
val_f1 = f1_score(y_val, val_preds, average='macro')
train_logloss = log_loss(y_train, train_probs)
val_logloss = log_loss(y_val, val_probs)

print("\n✅ XGBoost Model Performance (with Early Stopping):")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Training Log Loss  : {train_logloss:.4f}")
print(f"Validation Log Loss: {val_logloss:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")
print(f"Δ LogLoss Gap      : {abs(train_logloss - val_logloss):.4f}")

print("\nClassification Report (Validation Set):")
print(classification_report(y_val, val_preds))


TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

In [23]:
# %% [Stacking Ensemble Model - Enhanced]

from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, log_loss, classification_report

# --- Define Base Models (using tuned params if available) ---
base_models = [
    ("xgb", XGBClassifier(
        n_estimators=109,
        learning_rate=0.0504,
        max_depth=4,
        subsample=0.7587,
        colsample_bytree=0.8293,
        gamma=0.4669,
        min_child_weight=5,
        eval_metric="mlogloss",
        random_state=42,
        n_jobs=-1
    )),
    ("lgbm", LGBMClassifier(
        n_estimators=103,
        learning_rate=0.0158,
        max_depth=7,
        subsample=0.9337,
        colsample_bytree=0.9123,
        min_child_samples=73,
        random_state=42,
        n_jobs=-1
    )),
    ("cat", CatBoostClassifier(
        iterations=227,
        learning_rate=0.0557,
        depth=7,
        l2_leaf_reg=4.52,
        verbose=0,
        random_seed=42
    )),
    ("rf", RandomForestClassifier(
        n_estimators=747,
        max_depth=9,
        min_samples_split=6,
        min_samples_leaf=5,
        max_features=None,
        random_state=42,
        n_jobs=-1
    ))
]

# --- Define Meta (Blender) Model ---
meta_model = LogisticRegression(
    max_iter=2000,
    multi_class="multinomial",
    solver="lbfgs",
    random_state=42
)

# --- Build the Stacking Model ---
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1,
    passthrough=False
)

# --- Train Ensemble ---
stacking_model.fit(X_train, y_train)

# --- Evaluate ---
train_preds = stacking_model.predict(X_train)
val_preds = stacking_model.predict(X_val)
train_probs = stacking_model.predict_proba(X_train)
val_probs = stacking_model.predict_proba(X_val)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average='macro')
val_f1 = f1_score(y_val, val_preds, average='macro')
train_logloss = log_loss(y_train, train_probs)
val_logloss = log_loss(y_val, val_probs)

print("\n Stacking Ensemble Performance:")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Training Log Loss  : {train_logloss:.4f}")
print(f"Validation Log Loss: {val_logloss:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")
print(f"Δ LogLoss Gap      : {abs(train_logloss - val_logloss):.4f}")

print("\nClassification Report (Validation Set):")
print(classification_report(y_val, val_preds))


# --- Train Ensemble ---
stacking_model.fit(X_train, y_train)

# --- Evaluate ---
train_preds = stacking_model.predict(X_train)
test_preds = stacking_model.predict(X_test)

train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)
train_f1 = f1_score(y_train, train_preds, average="macro")
test_f1 = f1_score(y_test, test_preds, average="macro")

print("\n Stacking Ensemble Performance:")
print(f"Training Accuracy: {train_acc:.4f}")
print(f"Test Accuracy     : {test_acc:.4f}")
print(f"Training F1 Score : {train_f1:.4f}")
print(f"Test F1 Score     : {test_f1:.4f}")


c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



✅ Stacking Ensemble Performance:
Training Accuracy : 0.9553
Validation Accuracy: 0.9356
Training F1 Score  : 0.9379
Validation F1 Score: 0.9098
Training Log Loss  : 0.1380
Validation Log Loss: 0.1895
Δ F1 Gap           : 0.0281
Δ LogLoss Gap      : 0.0514

Classification Report (Validation Set):
              precision    recall  f1-score   support

           0       0.91      0.87      0.89       628
           1       0.95      0.97      0.96      2546
           2       0.91      0.86      0.89       457

    accuracy                           0.94      3631
   macro avg       0.92      0.90      0.91      3631
weighted avg       0.94      0.94      0.94      3631



KeyboardInterrupt: 

In [44]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

def tune_model_with_optuna(model_name, n_trials=30):
    # --- Split train into sub-train/validation ---
    X_subtrain, X_val, y_subtrain, y_val = train_test_split(
        X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
    )

    def objective(trial):
        # XGBoost
        if model_name == "xgb":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
                "max_depth": trial.suggest_int("max_depth", 3, 7),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0.0, 2.0),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 6),
                "n_jobs": -1,
                "random_state": 42,
                "eval_metric": "mlogloss",
                "use_label_encoder": False,
                "objective": "multi:softprob",
                "num_class": len(np.unique(y_train))
            }
            model = XGBClassifier(**params)

        # LightGBM
        elif model_name == "lgbm":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
                "max_depth": trial.suggest_int("max_depth", 3, 7),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
                "random_state": 42,
                "n_jobs": -1
            }
            model = LGBMClassifier(**params)

        # CatBoost
        elif model_name == "cat":
            params = {
                "iterations": trial.suggest_int("iterations", 100, 500),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
                "depth": trial.suggest_int("depth", 3, 7),
                "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
                "random_seed": 42,
                "verbose": 0
            }
            model = CatBoostClassifier(**params)

        # Random Forest
        elif model_name == "rf":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "max_depth": trial.suggest_int("max_depth", 2, 7),
                "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
                "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
                "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
                "n_jobs": -1,
                "random_state": 42
            }
            model = RandomForestClassifier(**params)

        # --- Train & Evaluate ---
        model.fit(X_subtrain, y_subtrain)
        val_preds = model.predict(X_val)

        val_f1 = f1_score(y_val, val_preds, average="macro")

        # Objective: directly maximize F1
        return val_f1

    # --- Run Optuna study ---
    study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # --- Results ---
    print(f"\n✅ Best {model_name.upper()} Params:")
    print(study.best_params)
    print(f"Best F1 Score: {study.best_value:.4f}")

    return study


In [ ]:
# XGBoost
# xgb_study = tune_model_with_optuna("xgb", n_trials=30) considerable

# LightGBM
# lgbm_study = tune_model_with_optuna("lgbm", n_trials=30) considerable 

# # CatBoost
# cat_study = tune_model_with_optuna("cat", n_trials=30)

# # Random Forest
# rf_study = tune_model_with_optuna("rf", n_trials=30)

[I 2025-10-30 20:03:21,067] A new study created in memory with name: no-name-014aba8a-bae4-4e19-ba08-d854081061e8


  0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:03:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-30 20:03:27,627] Trial 0 finished with value: 0.9170495113253788 and parameters: {'n_estimators': 250, 'learning_rate': 0.09556428757689246, 'max_depth': 6, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'gamma': 0.3119890406724053, 'min_child_weight': 1}. Best is trial 0 with value: 0.9170495113253788.


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:03:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

[I 2025-10-30 20:06:10,940] Trial 27 finished with value: 0.9177248371986106 and parameters: {'n_estimators': 436, 'learning_rate': 0.08583201625404041, 'max_depth': 4, 'subsample': 0.6285836974809043, 'colsample_bytree': 0.6736701896523487, 'gamma': 1.5146295322867214, 'min_child_weight': 4}. Best is trial 26 with value: 0.9200108436344022.


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:06:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-30 20:06:16,734] Trial 28 finished with value: 0.913451878606895 and parameters: {'n_estimators': 402, 'learning_rate': 0.04301447867794913, 'max_depth': 3, 'subsample': 0.722941617844659, 'colsample_bytree': 0.7512979163480473, 'gamma': 1.284535246652175, 'min_child_weight': 2}. Best is trial 26 with value: 0.9200108436344022.


c:\Users\ommah\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:06:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-30 20:06:21,579] Trial 29 finished with value: 0.9205029567214768 and parameters: {'n_estimators': 335, 'learning_rate': 0.06869744187596126, 'max_depth': 4, 'subsample': 0.5268577734910248, 'colsample_bytree': 0.6233187244227066, 'gamma': 1.7743397533828527, 'min_child_weight': 3}. Best is trial 29 with value: 0.9205029567214768.

✅ Best XGB Params:
{'n_estimators': 335, 'learning_rate': 0.06869744187596126, 'max_depth': 4, 'subsample': 0.5268577734910248, 'colsample_bytree': 0.6233187244227066, 'gamma': 1.7743397533828527, 'min_child_weight': 3}
Best F1 Score: 0.9205


In [ ]:
# ✅ Best XGB Params:
# {'n_estimators': 600, 'learning_rate': 0.09905399158869002, 'max_depth': 6, 'subsample': 0.6146009721997969, 'colsample_bytree': 0.9828472877209867, 'gamma': 1.5294297733838427, 'min_child_weight': 1}
# Best Objective Value: 0.0686

# ✅ Best LGBM Params:
# {'n_estimators': 316, 'learning_rate': 0.06533353663762796, 'max_depth': 7, 'subsample': 0.569746930326021, 'colsample_bytree': 0.6460723242676091, 'min_child_samples': 43}
# Best Objective Value: 0.0701

# ✅ Best CAT Params:
# {'iterations': 496, 'learning_rate': 0.055895437266264544, 'depth': 7, 'l2_leaf_reg': 8.50438821157523}
# Best Objective Value: 0.0693

# ✅ Best RF Params:
# {'n_estimators': 414, 'max_depth': 12, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2'}
# Best Objective Value: 0.0732


In [33]:
# %% [Stacking Ensemble Model - Tuned Base Models]

from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# --- Define Tuned Base Models ---
base_models = [
    ("xgb", XGBClassifier(
        n_estimators=600,
        learning_rate=0.09905399158869002,
        max_depth=6,
        subsample=0.6146009721997969,
        colsample_bytree=0.9828472877209867,
        gamma=1.5294297733838427,
        min_child_weight=1,
        random_state=42,
        n_jobs=-1,
        eval_metric="mlogloss",
        objective="multi:softprob",
        use_label_encoder=False,
        num_class=len(np.unique(y_train))
    )),
    ("lgbm", LGBMClassifier(
        n_estimators=316,
        learning_rate=0.06533353663762796,
        max_depth=7,
        subsample=0.569746930326021,
        colsample_bytree=0.6460723242676091,
        min_child_samples=43,
        random_state=42,
        n_jobs=-1
    )),
    ("cat", CatBoostClassifier(
        iterations=496,
        learning_rate=0.055895437266264544,
        depth=7,
        l2_leaf_reg=8.50438821157523,
        verbose=0,
        random_seed=42
    )),
    ("rf", RandomForestClassifier(
        n_estimators=414,
        max_depth=12,
        min_samples_split=6,
        min_samples_leaf=2,
        max_features='log2',
        random_state=42,
        n_jobs=-1
    ))
]

# --- Meta Model (Blender) ---
meta_model = LogisticRegression(
    C=0.3,
    max_iter=1000,
    multi_class="multinomial",
    solver="lbfgs",
    random_state=42
)

# --- Build Stacking Ensemble ---
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1,
    passthrough=False
)

# --- Train Ensemble ---
stacking_model.fit(X_train, y_train)

# --- Evaluate ---
train_preds = stacking_model.predict(X_train)
val_preds = stacking_model.predict(X_val)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average="macro")
val_f1 = f1_score(y_val, val_preds, average="macro")

print("\n✅ Stacking Ensemble Performance:")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")


c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



✅ Stacking Ensemble Performance:
Training Accuracy : 0.9756
Validation Accuracy: 0.9361
Training F1 Score  : 0.9659
Validation F1 Score: 0.9098
Δ F1 Gap           : 0.0561


In [ ]:
# %% [Stacking Ensemble Model - Tuned Base Models with Ridge Meta-Model]

from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# --- Define Tuned Base Models ---
base_models = [
    ("xgb", XGBClassifier(
        n_estimators=600,
        learning_rate=0.09905399158869002,
        max_depth=6,
        subsample=0.6146009721997969,
        colsample_bytree=0.9828472877209867,
        gamma=1.5294297733838427,
        min_child_weight=1,
        random_state=42,
        n_jobs=-1,
        eval_metric="mlogloss",
        objective="multi:softprob",
        use_label_encoder=False,
        num_class=len(np.unique(y_train))
    )),
    # ("lgbm", LGBMClassifier(
    #     n_estimators=316,
    #     learning_rate=0.06533353663762796,
    #     max_depth=7,
    #     subsample=0.569746930326021,
    #     colsample_bytree=0.6460723242676091,
    #     min_child_samples=43,
    #     random_state=42,
    #     n_jobs=-1
    # )),
    # ("cat", CatBoostClassifier(
    #     iterations=496,
    #     learning_rate=0.055895437266264544,
    #     depth=7,
    #     l2_leaf_reg=8.50438821157523,
    #     verbose=0,
    #     random_seed=42
    # )),
    # ("rf", RandomForestClassifier(
    #     n_estimators=414,
    #     max_depth=12,
    #     min_samples_split=6,
    #     min_samples_leaf=2,
    #     max_features='log2',
    #     random_state=42,
    #     n_jobs=-1
    # ))
]

# --- Meta Model (Ridge Classifier with CV) ---
meta_model = RidgeClassifierCV(
    alphas=np.logspace(-3, 3, 10),  # range of regularization strengths
    cv=5
)

# --- Build Stacking Ensemble ---
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1,
    passthrough=False
)

# --- Train Ensemble ---
stacking_model.fit(X_train, y_train)

# --- Evaluate ---
train_preds = stacking_model.predict(X_train)
val_preds = stacking_model.predict(X_val)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average="macro")
val_f1 = f1_score(y_val, val_preds, average="macro")

print("\n✅ Stacking Ensemble Performance (Ridge Meta-Model):")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")



✅ Stacking Ensemble Performance (Ridge Meta-Model):
Training Accuracy : 0.9861
Validation Accuracy: 0.9383
Training F1 Score  : 0.9805
Validation F1 Score: 0.9145
Δ F1 Gap           : 0.0661
